## Машинное обучение для работы с текстами


### Лемматизация

Лемматизация - приведение всех слов текста в начальную форму

Существует большое количество библиотек с поддержкой русского языка, мы рассмотрим библиотеку `pymorphy2`. В таких библиотеках находятся предобученные модели для лемматизации текста для русского и украинского языков.

https://pymorphy2.readthedocs.io/en/latest/

In [1]:
# Раскомментировать для установки
# !pip install pymorphy2

In [5]:
import pymorphy2

# Создадим анализатор морфем
analyser = pymorphy2.MorphAnalyzer()

analyser.parse('думать')

[Parse(word='хочу', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='хотеть', score=1.0, methods_stack=((DictionaryAnalyzer(), 'хочу', 3136, 1),))]

Метод `parse` возвращает массив объектов со следующими атрибутами:
-     **tag** -  набор граммем. В данном случае слово думать – это инфинитив глагола (INFN),  несовершенного вида (impf)
- **normal_form**– нормального форма слова;
- **score** – оценка вероятности того, что данный разбор правильный;
- **methods_stack** – тип словаря распарсенного слова с его индексом.

Объекты в массиве расположены в порядке убывания атрибута score, поэтому чаще всего следует брать 1й элемент. Получим нормальную форму глагола при помощи атрибута `normal_form`

In [3]:
analyser.parse('думал')[0].normal_form

'думать'

### Токенизация

Токенизация - разбивка текста на составляющие (слова, предложения и т.д.)

Токенизацию русского текста будем проводить при помощи библиотеки `nltk`. Функция `sent_tokenize` разделяет текст на предложения, `word_tokenize` на слова

https://www.nltk.org/

In [4]:
# Раскомментировать для установки
#!pip install nltk


In [4]:
import nltk

# Раскомментировать для установки
# nltk.download('punkt')
text = "Петя много знает, т.к. много читает. Хочу быть like Петя."

# Деление на предложения
token_sent = nltk.tokenize.sent_tokenize(text, language = 'russian')

# Деление на слова 
token_word = nltk.tokenize.word_tokenize(text, language = 'russian')


print("Текст, токенизированный по предложениям:", token_sent)
print()
print("Текст, токенизированный по словам:", token_word)

Текст, токенизированный по предложениям: ['Петя много знает, т.к. много читает.', 'Хочу быть like Петя.']

Текст, токенизированный по словам: ['Петя', 'много', 'знает', ',', 'т.к.', 'много', 'читает', '.', 'Хочу', 'быть', 'like', 'Петя', '.']


Как видно, `nltk` разбил текст на предложения, учитывая сокращение "т.к."


Напишем функцию для токенизации текста по словам и последующей его лемматизации и удаления знаков препинания

In [6]:
def preprocess_text(text, drop = False):
    analyser = pymorphy2.MorphAnalyzer()
    token_word = nltk.tokenize.word_tokenize(text, language = 'russian')
    
    # Создадим словарь из-за того, что операция проверки значения работает за O(1)
    punctuation_array = [',', '-', '.', ':', ';', '?', '!', '"', "'", '(', ')']
    punctuation_dict = {i : 0 for i in punctuation_array}
    
    preprocessed_text = []
    for word in token_word:
        if drop:
            if word not in punctuation_dict.keys():
                preprocessed_text.append(analyser.parse(word)[0].normal_form)
        else: 
            preprocessed_text.append(analyser.parse(word)[0].normal_form)
                    
    return preprocessed_text


Предобработаем стихотворение Александра Блока

In [7]:
text = """
Ночь, улица, фонарь, аптека,
Бессмысленный и тусклый свет.
Живи еще хоть четверть века —
Всё будет так. Исхода нет.

Умрешь — начнешь опять сначала
И повторится всё, как встарь:
Ночь, ледяная рябь канала,
Аптека, улица, фонарь."""


preprocess_text(text)

['ночь',
 ',',
 'улица',
 ',',
 'фонарь',
 ',',
 'аптека',
 ',',
 'бессмысленный',
 'и',
 'тусклый',
 'свет',
 '.',
 'живить',
 'ещё',
 'хоть',
 'четверть',
 'век',
 '—',
 'всё',
 'быть',
 'так.',
 'исход',
 'нет',
 '.',
 'умереть',
 '—',
 'начать',
 'опять',
 'сначала',
 'и',
 'повториться',
 'всё',
 ',',
 'как',
 'встарь',
 ':',
 'ночь',
 ',',
 'ледяной',
 'рябь',
 'канал',
 ',',
 'аптека',
 ',',
 'улица',
 ',',
 'фонарь',
 '.']

In [9]:
ord(preprocess_text(text)[18])

8212

In [10]:
ord('-')

45

## Данные и постановка задачи

Весь материал ниже будет показан на примере части данных из корпуса русских твитов https://study.mokoron.com/  http://www.swsys.ru/index.php?page=article&id=3962&lang=


База данных состоит из 12 столбцов:

- id: уникальный номер сообщения в системе twitter;
- tdate: дата публикации сообщения (твита);
- tmane: имя пользователя, опубликовавшего сообщение;
- ttext:  текст сообщения (твита);
- ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
- trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
- trtw: количество ретвитов;
- tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
- tstcount: число всех сообщений пользователя в сети twitter;
- tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);
- tfrien: количество друзей пользователя (те люди, которых читает пользователь);
- listcount: количество листов-подписок в которые добавлен твиттер-пользователь.


Наша задача предсказать тип твита: позитивный или негативный

In [11]:
import pandas as pd


# Читаем данные
data = pd.read_csv('twitter_corpus.csv')
data.sample(10)


,id,tdate,tmane,ttext,ttype,trep,tfav,trtw,tstcount,tfol,tfrien,listcount
3613,410304092859531264,1386659093,Lenookkk,"Лежу спокойно в кровати, и тут уведомление из ...",-1,0,0,0,5281,747,12,3
7668,410779530778136577,1386772446,ekaterinab172,"Мужчины храпят во сне, чтобы защитить своих от...",1,0,0,0,4,0,2,0
1538,410856223907393537,1386790731,ChernushoVa514,капец..как уснуть?это учащенное сердцебиение у...,-1,0,0,0,147,8,8,0
8979,410032963330768896,1386594451,dotdroid,@Oh_Philip17 @corpz_ @aka_opex так ты чо все т...,1,0,0,0,40694,441,190,15
5588,410796733912739840,1386776548,Strange_eternal,"@WaveOfSweetFire Хорошее желание * сказал я, ...",1,0,0,0,20803,600,688,11
520,412817869999968256,1387258424,TheMaDogg,xxx: Моя жизнь принадлежит Орде!!!! / ххх: а к...,-1,0,0,0,69216,3443,6815,8
4849,412193238062465024,1387109500,Love_Batman69,RT @Fereira1999: Люди блять вы охерели со всем...,-1,0,3,0,1692,1181,1186,6
6977,410866218976165888,1386793114,Raziiiiiiiii,"RT @karrimov: Кстати, GTA: San Andreas вышла в...",1,0,3,0,7641,160,936,1
9595,409046441630629888,1386359246,zkate97,@elizabettlapo а еще сумасшедшие танцы перед з...,1,0,0,0,725,50,46,0
6918,409773726994292736,1386532644,MaxDisk,@arvidOS @Alex_Shvarz @orion_575 та нет. Есть ...,1,0,0,0,117836,1755,638,146


## Регулярные выражения

В модуле Python вы уже работали с библиотекой `re` и регулярными выражениями. Если забыли, держите шпаргалку https://www.debuggex.com/cheatsheet/regex/python

**Как вы думаете, что можно выделить из текста, чтобы уменьшить размерность и при этом не потерять важной информации?**


In [19]:
import re
def parce_text(text):
    res = re.findall("['А-яёË']+", text)
    # Приведем в строку
    total_str = ''
    for word in res:
        total_str += word + ' '
    return total_str

# Как можно изменить функцию выше, чтобы парсить смайлики?
s = 'Хотела написать ванили ванильную но не вышло;('

parce_text(s)

'Хотела написать ванили ванильную но не вышло '

In [20]:
data['text_parsed'] = data['ttext'].apply(lambda x: parce_text(x))
data.sample(10)

,id,tdate,tmane,ttext,ttype,trep,tfav,trtw,tstcount,tfol,tfrien,listcount,text_parsed
2924,411792431701434369,1387013941,Flyyyy4,@milena_galk и я очень( сегодня приснилось про...,-1,0,0,1,8523,127,87,0,и я очень сегодня приснилось просто
6149,410300052088061952,1386658130,walter_tanyaZ,Я вчера нарисовал его :-) оцените пжалки http...,1,0,0,0,9232,271,216,5,Я вчера нарисовал его оцените пжалки
1882,413932462172299264,1387524164,a_ni_dam,Казавшийся нормистый фик скатился в дерьмо((,-1,0,0,0,12576,71,30,0,Казавшийся нормистый фик скатился в дерьмо
2212,413305023741710336,1387374571,vika30011999,"надо идти покупать вещи к новому году,лень:(",-1,0,0,0,3569,49,65,0,надо идти покупать вещи к новому году лень
1014,409845234114498560,1386549693,jekaterinaonu,Даня заболел :( каждый день что-то случается....,-1,0,0,0,886,51,61,0,Даня заболел каждый день что то случается
3787,415846621067231232,1387980535,v_kimo,Друзья как можно было написать трек ОТРЫВКИ ИЗ...,-1,0,0,0,1670,1141,112,2,Друзья как можно было написать трек ОТРЫВКИ ИЗ...
6136,410870321315450880,1386794092,ian_vladimirov,"Доставило: «В последний раз, когда создатель L...",1,0,0,0,2293,4061,102,43,Доставило В последний раз когда создатель Бред...
2107,413825881392828416,1387498753,jolliss_kierce,"Одиночество это когда в онлайне 50 друзей, а н...",-1,0,0,0,575,209,197,0,Одиночество это когда в онлайне друзей а напис...
6498,409061426582786048,1386362818,maskuznetsova,Сегодня игра ЦСКА против СКА понравилась! Всег...,1,0,0,0,1619,152,162,2,Сегодня игра ЦСКА против СКА понравилась Всегд...
5309,410380789801816064,1386677379,buzazuwedup,"— блин, просто сплошной праздник какой-то у на...",1,0,0,0,616,188,198,0,блин просто сплошной праздник какой то у нас в...


## Bag of words

Машины работают на языке математики, поэтому они не могут воспринимать текст в виде последовательности символов, которая понятна человеку. Переведем текст в формат более привычный для машины - вектора.

**Bag of words (мешок слов)** - переводит текст в вектор, учитывая частоту встречаемости в нем слов, но не учитывает их последовательность.

В случае когда текстов несколько bag of words перобразует их в матрицу, где строки это тексты, а столбцы - уникальные слова.

Рассмотрим на примере:

In [21]:
# Исходный текст
text = 'Ехал Грека через реку, видит Грека в реке рак, сунул грека руку в реку, рак за руку Греку цап!'

# Токенизируем и лемматизируем и текст
new_text = preprocess_text(text)
new_text

['ехать',
 'грек',
 'через',
 'река',
 ',',
 'видеть',
 'грек',
 'в',
 'река',
 'рак',
 ',',
 'сунуть',
 'грек',
 'рука',
 'в',
 'река',
 ',',
 'рак',
 'за',
 'рука',
 'грек',
 'цап',
 '!']

In [22]:
# Посчитаем кол-во вхождений каждого слова
from collections import Counter

cnt = Counter(new_text)
print(cnt)

# Выделим массив значений
bag_of_words = list(cnt.values())
print("Мешок слов:", bag_of_words)

Counter({'грек': 4, 'река': 3, ',': 3, 'в': 2, 'рак': 2, 'рука': 2, 'ехать': 1, 'через': 1, 'видеть': 1, 'сунуть': 1, 'за': 1, 'цап': 1, '!': 1})
Мешок слов: [1, 4, 1, 3, 3, 1, 2, 2, 1, 2, 1, 1, 1]


### Bag of words в sklearn

В библиотеке `sklearn` есть модуль `features_extraction.text`, предназначенный для предобработки текстовых данных https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

Для создания мешка слов для наших данных нам потребуется функция `CountVectorizer()` 

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

cnt = CountVectorizer()

bag_of_words_sklearn = cnt.fit_transform(data['ttext'].values)

print(bag_of_words_sklearn.shape)

(10000, 32637)


## TF-IDF

Для измерения важности слова в текса, являющегося частью корпуса применяется TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency). TF отвечает за упоминание слов в конкретном тексте, а IDF за частоту употребления слова во всем корпусе, таким образом слова, неважные для вообще всех документов, например, предлоги или междометия — получат очень низкий вес TF-IDF (потому что часто встречаются во всех-всех документах), а важные — высокий.

TF-IDF часто применяется в SEO оптимизации текстов

TF-IDF рассчитываетсся как: $TF-IDF = TF * IDF$; 

$$TF = \frac{c}{N}$$, где c - кол-во употребления слова, N - общее кол-во слов в тексте

$$IDF = \log_{a}{\frac{D}{d}}$$, где а - основание логарифма, выбирающееся от задачи, чаще всего полагают а = 2 или а = 10, D - общее кол-во текстов в корпусе, d - кол-во текстов, в которых употребляется слово


В библиотеке `sklearn` есть встроенная функция `TfidfVectorizer()` для подсчета TF-IDF. 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_cnt = TfidfVectorizer()

tfidf = tfidf_cnt.fit_transform(data['ttext'].values)

print(tfidf.shape)

(10000, 32637)


**Важно! Если данные разделены на тестовую и обучающую выборки, то Tfidf стоит запускать только на обучающей выборке**

## Эмбеддинги слов

Слова в предложении сами по себе не имеют смысла, она заключается в контексте, свойствах и т.д. Выше при переводе слова в вектор это не учитывалось. 

При работе с нетабличными данными, их переводят **в векторное представление**, эмбеддинги слов (word embeddings) - частный случай таких представлений.

Эмбеддинги содержат в себе информацию о соотношении слов в тексте и их свойствах, например свойства слова "пожарный": "мужчина", "герой", "огонь" и т.д.

Эти данные нужны для определения схожести векторных представлений слов. Как вы помните, схожесть векторов определяется расстоянием между ними. Например, мы интуитивно понимаем, что слова "лопата" и "вилы" гораздо более схожи, чем слова "дробовик" и "валентинка", они употребляются в похожем контексте

То как получаются такие векторы - за пределами нашего курса, материалы по этой теме: https://habr.com/ru/company/ods/blog/329410/

## Классификация твитов

Вернемся к задачи классификации твитов, в качесте признаков выступают слова в корпусе и их TF-IDF. Обучим логистическую регрессию

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf, data['ttype'], test_size = 0.2, random_state = 42)



In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()

model.fit(X_train, y_train)
pred = model.predict(X_test)

print('Accuracy:', accuracy_score(pred, y_test))

Accuracy: 0.68
